# Abstract Syntax Trees

In [1]:
import ast
tree = ast.parse('x = 3; print x')
tree

In [2]:
code = compile(tree, filename='', mode='exec')
code

<code object <module> at 0000000004353930, file "", line 1>

In [3]:
exec(code)

3


An AST is just a tree with nodes. Let's see how that's like...

In [4]:
ast.dump(tree)

"Module(body=[Assign(targets=[Name(id='x', ctx=Store())], value=Num(n=3)), Print(dest=None, values=[Name(id='x', ctx=Load())], nl=True)])"

In [5]:
tree.body

[<_ast.Assign at 0x43749e8>, <_ast.Print at 0x4374b38>]

In [6]:
for node in ast.walk(tree):
    print node

In [7]:
tree.body[0] = ast.Assign(targets=[ast.Name(id='x', ctx=ast.Store())], value=ast.Num(n=5))

In [8]:
ast.dump(tree)

"Module(body=[Assign(targets=[Name(id='x', ctx=Store())], value=Num(n=5)), Print(dest=None, values=[Name(id='x', ctx=Load())], nl=True)])"

In [9]:
exec(compile(tree, filename='', mode='exec'))

TypeError: required field "lineno" missing from stmt

To compile an AST, every node must have `lineno` and `col_offset` attributes. Nodes produced by parsing regular code already have these, but nodes you create programmatically don’t. We'll use `ast.fix_missing_locations` to fix this.

In [10]:
exec(compile(ast.fix_missing_locations(tree), filename='', mode='exec'))

5


In [11]:
class RewriteName(ast.NodeTransformer):

    def visit_Name(self, node):
        return ast.copy_location(ast.Name(id='y', ctx=ast.Store()), node)

tree = RewriteName().visit(tree)

In [12]:
import astunparse

for line in astunparse.unparse(tree).split('\n'):
    print line


y = 5
print y

